In [1]:
import pandas as pd
import os.path as Path
import numpy as np
import string as str

In [2]:
%load_ext autoreload
%autoreload 2
from data_bpm.interface.main import *
from data_bpm.ml_logic.preprocessor import *

In [3]:
data_for_ml = pd.read_csv('../raw_data/data_for_ml.csv',index_col=0)
data_for_ml

,description,headline,location,company,jobTitle,jobDescription,jobLocation,jobDateRange,jobDuration,company2,...,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,Attendance
UserID,,,,,,,,,,,,,,,,,,,,,
512.0,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",orbis systems,"General Manager, Orbis International Technolog...",A highly driven professional leading the US op...,"Carrollton, Texas, United States",May 2022 - Present,1 yr 10 mos,Texas Instruments,...,21.0,Strategic Planning,21.0,Forecasting,21.0,Organizational Development,21.0,Business Strategy,21.0,0
652.0,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",holidu,Tech product support intern,NaN,"Munich, Bavaria, Germany · Hybrid",Oct 2023 - Present,5 mos,World Simplified UG,...,NaN,Team Leadership,NaN,Cross-functional Team Leadership,1.0,Project Planning,21.0,Business Development,NaN,0
618.0,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",a.t.e.m. management gmbh,Web Designer,Skills: Web Design · Cascading Style Sheets (C...,"Berlin, Germany",Aug 2022 - Present,1 yr 7 mos,Freelance,...,NaN,JavaScript,2.0,Cascading Style Sheets (CSS),2.0,Front-End Development,1.0,Wireframing,1.0,0
96.0,NaN,Product & Growth,"Berlin, Berlin, Germany",clink,Chief Experience Officer,NaN,"Berlin, Germany",Sep 2023 - Present,6 mos,OptioPay Group (now Clink),...,40.0,Cross-functional Team Leadership,3.0,Business Development,33.0,Mobile Applications,10.0,Mobile Technology,35.0,0
194.0,As a product marketing manager at the JetBrain...,Product marketing manager at JetBrains | Mentor,"Berlin, Berlin, Germany",jetbrains,Product Marketing Manager,NaN,"Berlin, Germany",Oct 2021 - Present,2 yrs 5 mos,Product Marketing Alliance,...,8.0,Marketing Communications,8.0,Customer Insight,13.0,Market Analysis,12.0,Marketing Research,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229.0,As a Senior Product Manager with over 12+ year...,Product Manager @ KAYAK,"Berlin, Berlin, Germany",kayak,Product Manager,NaN,"Berlin, Germany · On-site",Nov 2023 - Present,5 mos,Sastaticket.pk,...,1.0,Customer Onboarding,1.0,Mentoring,1.0,Product Development,1.0,Analytical Skills,NaN,1
461.0,A lady with a previous marketing background an...,Junior Product Designer✨| UX/UI | Figma | Rese...,"Berlin, Berlin, Germany",velents,Junior Product Designer,Velents AI is a hiring platform that will help...,"Berlin, Germany · Remote",May 2023 - Present,11 mos,Brainnest,...,NaN,Prototyping,NaN,User Research,NaN,User Experience Design (UED),NaN,Design Systems,21.0,1
10.0,"A passionate Product Leader Focused on People,...",Product Leader & Mentor • Lead Product Manager...,"Berlin, Berlin, Germany",spryker,Lead Product Manager,Skills: Product Management · Team Leadership ·...,"Berlin, Germany · Remote",Aug 2022 - Present,1 yr 8 mos,Spryker,...,2.0,Problem Solving,41.0,Team Management,NaN,Technical Understanding,31.0,Product Road Mapping,3.0,1


In [4]:
X_processed = preprocess_features(data_for_ml.drop(columns=['Attendance']), save_pipeline = True)


✅ MetadataTransformer constructor
✅ MetadataTransformer constructor
✅ MetadataTransformer is called for the first time
✅ X_processed, with shape (474, 34)
[[1.         0.07942238 0.48854962 ... 0.54997591 0.21428154 0.69186946]
 [1.         0.01805054 0.01908397 ... 0.1908353  0.43886729 0.33560341]
 [1.         0.06859206 0.12977099 ... 0.49946877 0.585145   0.29062143]
 ...
 [1.         0.07220217 0.02671756 ... 0.39042567 0.58771065 0.39970056]
 [1.         0.07581227 0.         ... 0.54954423 0.45180968 0.39370713]
 [1.         0.03249097 0.07251908 ... 0.32774429 0.08364515 0.49241546]]

Saving preprocessing pipeline to local disk...
✅ Preprocessing pipeline saved locally


In [5]:
y_train = data_for_ml['Attendance']

In [14]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Assuming you have your data in X_train, X_test, y_train, y_test

# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

# Define parameter grids for RandomizedSearchCV and GridSearchCV
param_grid_random = {
    'n_estimators': randint(290, 360),
    'learning_rate': uniform(0.3, 0.5),
    'max_depth': randint(3, 7),
    'min_child_weight': randint(1, 5),
    'subsample': uniform(0.5, 0.8),
    'colsample_bytree': uniform(0.6, 0.7),
    'gamma': uniform(0.3, 0.6),
}


# Perform RandomizedSearchCV
rand_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid_random, n_iter=1000, scoring='roc_auc', cv=2, verbose=1, n_jobs=-1)
rand_search.fit(X_processed, y_train)
print("Best parameters found by RandomizedSearchCV:", rand_search.best_params_)
print("Best AUC score found by RandomizedSearchCV:", rand_search.best_score_)


Fitting 2 folds for each of 1000 candidates, totalling 2000 fits
Best parameters found by RandomizedSearchCV: {'colsample_bytree': 0.773041181465276, 'gamma': 0.4250180588095578, 'learning_rate': 0.7633999035607152, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 311, 'subsample': 0.7897398402939455}
Best AUC score found by RandomizedSearchCV: 0.5738635633372475


/Users/yulka/.pyenv/versions/3.10.6/envs/data-bpm/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1286 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/yulka/.pyenv/versions/3.10.6/envs/data-bpm/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/yulka/.pyenv/versions/3.10.6/envs/data-bpm/lib/python3.10/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/Users/yulka/.pyenv/versions/3.10.6/envs/data-bpm/lib/python3.10/site-packa

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
Best parameters found by RandomizedSearchCV: {'colsample_bytree': 0.7981976920904925, 'gamma': 0.1493851548677918, 'learning_rate': 0.2873836203172824, 'max_depth': 4, 'min_child_weight': 9, 'n_estimators': 202, 'subsample': 0.981672482605199}
Best AUC score found by RandomizedSearchCV: 0.5497622334713913



In [7]:

param_grid_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 4, 9],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'gamma': [0, 0.1, 0.2],
}
# Perform GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_grid, scoring='roc_auc', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_processed, y_train)
print("Best parameters found by GridSearchCV:", grid_search.best_params_)
print("Best AUC score found by GridSearchCV:", grid_search.best_score_)

# Use the best model from GridSearchCV to predict probabilities
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_processed)[:, 1]




Fitting 5 folds for each of 2187 candidates, totalling 10935 fits
Best parameters found by GridSearchCV: {'colsample_bytree': 0.8, 'gamma': 0.2, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.7}
Best AUC score found by GridSearchCV: 0.5470974952337124


NameError: name 'y_test' is not defined

In [8]:
# Calculate AUC score on the test set
auc_score = roc_auc_score(y_train, y_pred_proba)
print("AUC score on the test set:", auc_score)

AUC score on the test set: 1.0


In [14]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.4 MB/s eta 0:00:0000:0100:01
